In [94]:
import pandas as pd
import numpy as np
import glob

In [95]:
zo = pd.DataFrame()
for file in glob.glob("dbm*.csv"):
    df = pd.read_csv(file)
    zo = zo.append(df, ignore_index=True)

In [96]:
dbm = pd.DataFrame()
for file in glob.glob("att*.csv"):
    df = pd.read_csv(file, skipinitialspace=True, 
            usecols=["Insertion Order", "Insertion Order ID", "Line Item", "Line Item ID", "Revenue (Adv Currency)"])
    dbm = dbm.append(df, ignore_index=True)

In [97]:
IOIDmap = pd.read_csv("map.csv", usecols=["IOID", "Targeting"])

In [98]:
dbm.columns = ["IO", "IOID", "LI", "LIID", "Spend"]

In [99]:
dbm = dbm.drop(dbm.index[dbm.tail(5).index.values])
dbm = dbm.loc[dbm.LI.isnull() == False]

In [74]:
dbm.IOID = dbm.IOID.astype("int64")
dbm.LIID = dbm.LIID.astype("int64")

In [75]:
#pd.DataFrame(dbm.IOID.unique(), columns=["uniqueIOID"]).to_csv(
#    "D:\\Users\\ylin\\Desktop\\Aviva ATT output\\"+"uniqueIOID.csv", index=False)

In [76]:
dbm_ = pd.DataFrame(dbm.pivot_table(values="Spend", index=["IOID", "IO"], aggfunc=np.sum)).reset_index()

In [43]:
zo_ = pd.DataFrame(zo.pivot_table(values="zo_attr", index="dbm_insertion_order_id", aggfunc=np.sum)).reset_index()

In [44]:
zo_.columns = ["IOID", "zo_attr"]

In [80]:
data = dbm_.merge(zo_, on="IOID", how="left")

In [81]:
data = data.merge(IOIDmap, on="IOID", how="left")

In [85]:
data_ = data.pivot_table(values=["Spend", "zo_attr"], aggfunc=np.sum, index="Targeting").reset_index()

In [90]:
data_["zoCPQ"] = data_.Spend / data_.zo_attr

In [49]:
data_.to_csv("D:\\Users\\ylin\\Desktop\\"+"latest CPQ.csv", index=False)

In [91]:
data_

,Targeting,Spend,zo_attr,zoCPQ
0,Prospecting,128433.341320,15844.730934,8.105745
1,Remarketing existing,17893.703917,1943.679811,9.206097
2,Remarketing non-existing,186119.893788,29545.800527,6.299369
